In [47]:
import influxdb_client
import pandas as pd
import matplotlib.pyplot as plt

In [48]:
token = "lYtO30KjJmXRezB7nJaWbhUYHV7z9wKAtezFOmNyUtyakGE_Tlln5gMsWSwFy0S-lQSiqGtvhmRE3X7bbEABZw=="
org = "sgh"
url = "http://0.0.0.0:8086"

In [49]:
client = influxdb_client.InfluxDBClient(url=url, token=token, org=org)

In [50]:
query_api = client.query_api()

In [53]:
query = """from(bucket: "aeroponics")
  |> range(start: 2024-11-01T10:21:00Z)
  |> filter(fn: (r) => r["_measurement"] == "first")
  |> filter(fn: (r) => r["_field"] == "humidity")
  |> aggregateWindow(every: 1m, fn: mean, createEmpty: false)
  |> yield(name: "mean")"""
tables = query_api.query(query, org="sgh")

In [54]:
data = []
for table in tables:
    for record in table.records:
        data.append({
            "datetime": record.get_time(),
            "airTemperature": record.get_value(),
        })
df = pd.DataFrame(data)
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.set_index('datetime').asfreq("1min")
df

,airTemperature
datetime,
2024-12-16 05:13:00+00:00,58.327500
2024-12-16 05:14:00+00:00,52.215333
2024-12-16 05:15:00+00:00,49.499667
2024-12-16 05:16:00+00:00,53.409500
2024-12-16 05:17:00+00:00,46.871167
2024-12-16 05:18:00+00:00,49.645667
2024-12-16 05:19:00+00:00,44.098167
2024-12-16 05:20:00+00:00,44.675833
2024-12-16 05:21:00+00:00,47.833167


In [9]:
import csv
import os

# Define the header and data
header = ["mae", "mape", "rmse"]
data = [0.31, 0.1, 0.2]

# Check if the file exists and has a header
file_exists = os.path.isfile('metrix_log.csv')
header_exists = False

if file_exists:
    with open('metrix_log.csv', 'r') as file:
        reader = csv.reader(file)
        # Check the first row to see if it's the header
        header_exists = any(row == header for row in reader)

# Open the file in append mode
with open('metrix_log.csv', 'a', newline='') as file:
    writer = csv.writer(file)
    # Write the header if it doesn't exist
    if not header_exists:
        writer.writerow(header)
    # Write the data
    writer.writerow(data)


In [2]:
import pickle

model = pickle.load(open("model/XGBoost_airTemperature.pkl", 'rb'))
type(model)

xgboost.sklearn.XGBRegressor

In [5]:
from index import AdaptiveControll

adaptiveControll = AdaptiveControll()

df = adaptiveControll.predict()
df

,airTemperature,humidity
datetime,,
2024-12-24 16:57:23,25.170654,101.503448
2024-12-24 16:58:23,25.170654,101.503448
2024-12-24 16:59:23,25.170654,101.503448
2024-12-24 17:00:23,25.208376,101.515755
2024-12-24 17:01:23,25.208376,101.515755
...,...,...
2024-12-25 16:52:23,26.169426,95.026764
2024-12-25 16:53:23,26.169426,95.026764
2024-12-25 16:54:23,26.169426,95.026764


In [17]:
delay = adaptiveControll.FuzzyLogicNutrientPump(df.iloc[index]['airTemperature'], df.iloc[index]['humidity'])

1440

In [76]:
import pandas as pd

def compute() -> pd.DataFrame:
    delay_arr = []
    for index in range(len(df)):
        delay = adaptiveControll.FuzzyLogicNutrientPump(df.iloc[index]['airTemperature'], df.iloc[index]['humidity'])

compute()

1440
